In [ ]:
import json
import csv
import numpy as np
import pandas as pd

from sklearn.model_selection    import cross_validate,KFold, train_test_split
from sklearn.metrics            import classification_report, confusion_matrix,ConfusionMatrixDisplay
from sklearn.ensemble           import RandomForestClassifier,BaggingClassifier, IsolationForest, ExtraTreesClassifier,AdaBoostClassifier, GradientBoostingClassifier

from sklearn.feature_selection  import mutual_info_classif,f_classif, SelectKBest,RFE,SelectFromModel


# pip install xgboost
from sklearn.datasets           import make_classification
from sklearn.tree               import DecisionTreeClassifier
from xgboost                    import XGBClassifier
from sklearn.neighbors          import KNeighborsClassifier
from sklearn.naive_bayes        import GaussianNB


from sklearn.preprocessing      import MinMaxScaler, StandardScaler, Normalizer, Binarizer, minmax_scale, MaxAbsScaler, RobustScaler, QuantileTransformer, PowerTransformer


from sklearn.covariance         import EllipticEnvelope
from sklearn.neighbors          import LocalOutlierFactor
from sklearn.svm                import OneClassSVM, SVC



from sklearn.linear_model       import LogisticRegression
from sklearn.pipeline           import Pipeline

import sqlalchemy
from urllib.parse import quote  

In [ ]:
pd.set_option('display.max_columns', None)
# pd.set_option("max_rows", None)

In [ ]:
data = pd.read_csv(f'/content/drive/MyDrive/DATA/MetaData/Final/DS1.csv.gz',compression='gzip',lineterminator='\n');

# data=pd.concat([data[data["class"]=="bot"].sample(100),data[data["class"]=="human"].sample(100)])
data=data.sample(frac=1).reset_index(drop=True)


columnsList=data.columns.difference(["class"])

X = data[columnsList]
y = data['class'].map({"bot": 1, "human": 0});

FileNotFoundError: ignored

In [ ]:
def evaluating_model(fs, nm,model,X, y):

    pipe=Pipeline(steps=[('fs',fs),('sc',nm),('model', model)])

    
    scores = cross_validate(
        pipe,
        X,
        y,
        cv=KFold(n_splits=5, random_state=1, shuffle=True),
        scoring=["accuracy",'f1','recall','roc_auc','precision'],
        return_train_score=True,
        error_score='raise',
        return_estimator=True
    )

    SelectedFeature=list()
    for value in scores["estimator"]:
        if type(value.named_steps["fs"]).__name__=="RFE":
            SelectedFeature.append(columnsList[value.named_steps["fs"].support_].tolist())
        elif type(value.named_steps["fs"]).__name__=="SelectKBest" or type(value.named_steps["fs"]).__name__=="SelectFromModel":
            SelectedFeature.append(columnsList[value.named_steps["fs"].get_support()].tolist())
            # print(value.named_steps["fs"].get_feature_names_out())

    ffData=pd.DataFrame(scores).drop('estimator', axis=1)
    
    
    return scores["estimator"],SelectedFeature,ffData, ffData.agg({
      'fit_time':'sum', 
      'score_time':'sum',
      'test_accuracy':'mean',
      'train_accuracy':'mean',
      'test_f1':'mean',
      'train_f1':'mean',
      'test_recall':'mean',
      'train_recall':'mean',
      'test_roc_auc':'mean',
      'train_roc_auc':'mean',
      'test_precision':'mean',
      'train_precision':'mean',
    })